In [29]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math
import torch
import torch.nn as nn
from tqdm import tqdm
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
# plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
hidden_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
print(device)

cuda


In [3]:
MAX_LENGTH=100
SOS_token = 0
EOS_token = 1
teacher_forcing_ratio = 0.5

In [4]:
lines = open('../data/tmp2.txt',encoding="utf-8")\
    .read().strip().split('\n')
print(lines[:2])
print(len(lines))

['businessman canada executive noble r several u|agent,person,businessperson', 'additional allow also beneath capacity craft direct e east effectively elderly fail failure first four full however join launch move one orbit provide reach replace replacement return run satellite since state test would|place,satellite,artificialsatellite']
342781


In [5]:
pairs = [l.split('|') for l in lines]
print(pairs[:2])

[['businessman canada executive noble r several u', 'agent,person,businessperson'], ['additional allow also beneath capacity craft direct e east effectively elderly fail failure first four full however join launch move one orbit provide reach replace replacement return run satellite since state test would', 'place,satellite,artificialsatellite']]


In [6]:
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [8]:
pairs = filterPairs(pairs)
print(len(pairs)/len(lines))

0.9785343995145589


In [9]:
class Lang:
    def __init__(self, name, sep=" "):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
        self.sep = sep
    def addSentence(self, sentence):
        for word in sentence.split(self.sep):
            self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
input_lang = Lang('sentence'," ")
output_lang = Lang('level',",")
for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words) #9+70+219+2(start,end)

sentence 41306
level 300


In [11]:
pairs[:2]

[['businessman canada executive noble r several u',
  'agent,person,businessperson'],
 ['additional allow also beneath capacity craft direct e east effectively elderly fail failure first four full however join launch move one orbit provide reach replace replacement return run satellite since state test would',
  'place,satellite,artificialsatellite']]

In [12]:
random.choice(pairs)

['abandon along also bed cross crum depot downtown exit expressway extend find former highway interstate north northeast northern old railroad roadway rout route run south southern state street terminus toward u unusual way west western',
 'place,routeoftransportation,road']

In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device
                           
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
def indexesFromSentence(lang, sentence, sep=' '):
    return [lang.word2index[word] for word in sentence.split(sep)]


def tensorFromSentence(lang, sentence, sep=' '):
    indexes = indexesFromSentence(lang,sentence,sep)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0],' ')
    target_tensor = tensorFromSentence(output_lang, pair[1],',')
    return (input_tensor, target_tensor)

In [15]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [16]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [18]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [39]:
def score(encoder,decoder):
    count=0
    cl=[0]*3
#     num = len(pairs)
    num = 10000
#     for i in tqdm(range(len(pairs))):
    for i in tqdm(range(num)):
        pair=pairs[i]
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        tmp = pair[1].split(',')
        output_sentence = ",".join(output_words[:-1])
        if output_sentence==pair[1]:
            count+=1
        for j in range(len(tmp)):
            if tmp[j]==output_words[j]:
                cl[j]+=1
    print('Total Accuracy: ',count/num)
    print('L1 Accuracy: ',cl[0]/num)
    print('L2 Accuracy: ',cl[1]/num)
    print('L3 Accuracy: ',cl[2]/num)

In [21]:
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

  7%|▋         | 5005/75000 [05:19<54:25, 21.44it/s]  

5m 23s (- 75m 31s) (5000 6%) 2.3433


 13%|█▎        | 10001/75000 [10:41<1:05:30, 16.54it/s]

10m 46s (- 70m 1s) (10000 13%) 1.5986


 20%|██        | 15002/75000 [16:00<1:15:34, 13.23it/s]

...

69m 20s (- 10m 40s) (65000 86%) 0.3739


 93%|█████████▎| 70002/75000 [1:14:39<05:00, 16.61it/s]

74m 44s (- 5m 20s) (70000 93%) 0.3374


100%|██████████| 75000/75000 [1:20:02<00:00, 15.62it/s]

80m 7s (- 0m 0s) (75000 100%) 0.3416


In [22]:
evaluateRandomly(encoder1, attn_decoder1)

> appearance approach ascot career classic colt contender curragh dawn end first go highly lead march national newmarket old palace race rat royal season seven st thoroughbred three two win year
= species,horse,racehorse
< species horse racehorse <EOS>

> four gait lacrosse men orange perennial player team time university
= agent,athlete,lacrosseplayer
< agent athlete basketballplayer <EOS>

> album blunderbuss debut difficult gun need obtain permanently piece separate singer stock turn variation way
= work,musicalwork,album
< work musicalwork album <EOS>

> bulbul evergreen family found mid ochraceous songbird specie usually
= species,animal,bird
< species animal crustacean <EOS>

> affiliation approximately candidate caucus common conservative district eight election federal field finish house left lose may party pembina peter place politician previous progressive ran seat third win without
= agent,politician,memberofparliament
< event societalevent election <EOS>

> born golf golfer

In [44]:
t1 = time.time()
score(encoder1,attn_decoder1)
print(time.time()-t1)

100%|██████████| 10000/10000 [01:03<00:00, 157.07it/s]

Total Accuracy:  0.7431
L1 Accuracy:  0.9357
L2 Accuracy:  0.8437
L3 Accuracy:  0.7555
63.667301416397095


In [32]:
import matplotlib
output_words, attentions = evaluate(
    encoder1, attn_decoder1,
    "capital flagship network new serve south southern station television territory win")
matplotlib.use('TkAgg')
plt.matshow(attentions.numpy())
plt.savefig('test1.png')
plt.show()

In [42]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)



input = capital flagship network new serve south southern station television territory win
output = agent broadcaster televisionstation <EOS>


/home/yui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  # Remove the CWD from sys.path while we load stuff.
/home/yui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  # This is added back by InteractiveShellApp.init_path()


In [43]:
evaluateAndShowAttention(\
    "capital flagship network new serve south southern station television territory win")
plt.savefig('test2.png')

input = capital flagship network new serve south southern station television territory win
output = agent broadcaster televisionstation <EOS>


/home/yui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  # Remove the CWD from sys.path while we load stuff.
/home/yui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  # This is added back by InteractiveShellApp.init_path()
